這邊有一個資料集是我去識別後的 POS data
總共四個資料集

basket_item 被分成好幾個因為比較大

請大家嘗試把四個資料及合併並嘗試回答以下問題：
找到最佳的產品與顧客組合能帶給我最高的 revenue

此 side-project 並非現階段重要任務，目的是要讓大家熟悉 pandas, polars （）, chaining
請大家找時間練習，把你發現的竅門跟心得記錄下來做為教材
若太難我們可以將其難度降低，請隨時回饋問題或題目調整
此資料即將作為 pandas 最後章節的應用實戰

### 找到最佳的產品與顧客組合能帶給我最高的 revenue


In [21]:
# for data cleaning and processing
import numpy as np
import pandas as pd
import datetime as dt

# for drawing
import seaborn as sns
import matplotlib.pyplot as plt

# for ignoring warning messages
import warnings
warnings.filterwarnings('ignore')

In [22]:
import pandas as pd
import time
start_time = time.time()
df_basket_items = (
    pd.read_csv('basket_items_modified_4月.csv')
    .append(pd.read_csv('basket_items_modified_5月.csv'), ignore_index=True)
    .append(pd.read_csv('basket_items_modified_6月.csv'), ignore_index=True)
    .append(pd.read_csv('basket_items_modified_7月.csv'), ignore_index=True)
    .append(pd.read_csv('basket_items_modified_8月.csv'), ignore_index=True)
    .append(pd.read_csv('basket_items_modified_9月.csv'), ignore_index=True)
)

df_credit_card = pd.read_csv('credict_card_modified.csv')
df_ele_data = pd.read_csv('ele_data_modified.csv')
df_member_card = pd.read_csv('member_card_modified.csv')


In [ ]:
df_basket_items.head()

### 資料清理與過濾
##### 處理df_basket_items

In [27]:
import time
start_time = time.time()
def cleaning(df):
    df = (
        df.dropna()  # 移除空值
        .drop_duplicates() if df.duplicated().any() else df  # 如果有重複值就移除重複值
    )
    return df

df_basket_items = (
    cleaning(df_basket_items)
    .drop(['門市代碼_', '機台_', '序號_'], axis=1)  # 移除不需要的欄位
)
df_credit_card = cleaning(df_credit_card)


# df_basket_items = (
#     df_basket_items
#     .dropna()  # 移除空值
#     .drop_duplicates() if df_basket_items.duplicated().any() else df_basket_items  # 如果有重複值就移除重複值
#     .drop(['門市代碼_', '機台_', '序號_'], axis=1)  # 移除不需要的欄位
# )


### 資料轉換

In [28]:
df_basket_items = (
    df_basket_items
    .assign(銷售日期=pd.to_datetime(df_basket_items['銷售日期']))
    .astype({'門市代碼':'object','機台':'object','序號':'object','貨號':'object','條碼':'object'})
)

In [ ]:
df_basket_items.info()

In [ ]:
# 檢查重複
df_basket_items.duplicated().value_counts()

In [ ]:
# #把重複地拿掉
# df_basket_items.drop_duplicates(inplace=True)

In [ ]:
#df_basket_items.loc[df_basket_items.duplicated()]

In [ ]:
df_basket_items.describe()

In [ ]:
df_basket_items.describe(include = 'object')

In [ ]:
df_basket_items.head()

In [ ]:
df_basket_items.info()

In [ ]:
df_commodity_count = df_basket_items.groupby('短品名')['銷售數量'].sum().reset_index().sort_values(by='銷售數量',ascending=False)
df_commodity_count.head(20)

In [ ]:
df_commodity_price = df_basket_items.groupby('短品名')['總金額'].sum().reset_index().sort_values(by='總金額',ascending=False)
df_commodity_price.head(20)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid",{"font.sans-serif":['Microsoft JhengHei']})

# 使用groupby計算'銷售數量'加總
df_grouped = df_basket_items.groupby('短品名')['銷售數量'].sum().reset_index().sort_values(by='銷售數量',ascending=False)
df_grouped_top20 = df_grouped.head(20)
# 使用Seaborn繪製條形圖
sns.barplot(x='短品名', y='銷售數量', data=df_grouped_top20)

# 設定圖表標題和軸標籤
plt.title('銷售數量加總')
plt.xlabel('短品名')
plt.ylabel('銷售數量')

# 轉換X軸標籤為垂直方向
plt.xticks(rotation=90)
# 調整圖表大小，避免標籤重疊
plt.tight_layout()

# 顯示圖表
plt.show()

In [ ]:
sns.set_style("whitegrid",{"font.sans-serif":['Microsoft JhengHei']})

# 使用groupby計算'銷售數量'加總
df_grouped = df_basket_items.groupby('短品名')['總金額'].sum().reset_index().sort_values(by='總金額',ascending=False)
df_grouped_top20 = df_grouped.head(20)
# 使用Seaborn繪製條形圖
sns.barplot(x='短品名', y='總金額', data=df_grouped_top20)

# 設定圖表標題和軸標籤
plt.title('銷售金額加總')
plt.xlabel('短品名')
plt.ylabel('總金額')

# 轉換X軸標籤為垂直方向
plt.xticks(rotation=90)
# 調整圖表大小，避免標籤重疊
plt.tight_layout()

# 顯示圖表
plt.show()

### 處理df_credit_card 格式

In [ ]:
# 檢查空值
df_credit_card.isna().any()

In [ ]:
df_credit_card = (
    df_credit_card
    .assign(銷售日期 = pd.to_datetime(df_credit_card['銷售日期']),
            時間 = pd.to_datetime(df_credit_card['時間']).dt.strftime('%H:%M'))
    .astype({'項次':'object',
            '機台':'object',
            '序號':'object',
            '授權碼':'object',
            '調閱編號':'object',
            '卡號':'object',
            '員工代號':'object'})
)


# df_credit_card['項次']=df_credit_card['項次'].astype('object')
# df_credit_card['銷售日期']=pd.to_datetime(df_credit_card['銷售日期'])
# df_credit_card['機台']=df_credit_card['機台'].astype('object')
# df_credit_card['序號']=df_credit_card['序號'].astype('object')
# df_credit_card['時間']=pd.to_datetime(df_credit_card['時間'])
# df_credit_card['時間']=df_credit_card['時間'].dt.strftime('%H:%M')
# df_credit_card['授權碼']=df_credit_card['授權碼'].astype('object')
# df_credit_card['調閱編號']=df_credit_card['調閱編號'].astype('object')
# df_credit_card['卡號']=df_credit_card['卡號'].astype('object')
# df_credit_card['員工代號']=df_credit_card['員工代號'].astype('object')


In [ ]:
df_credit_card.info()

In [ ]:
# 檢查重複
df_credit_card.duplicated().value_counts()

In [ ]:
df_credit_card.describe(include = 'object')

### 處理ele_data 格式

### 處理member_data 格式

------

## 嘗試合併表格

In [ ]:
df_basket_items[(df_basket_items['序號']==3069) & (df_basket_items['銷售日期']=='2018-04-02')& (df_basket_items['機台']==66)]


In [ ]:
df_credit_card[(df_credit_card['序號']==3069) & (df_credit_card['銷售日期']=='2018-04-02')]

處理ele_data

In [ ]:
df_basket_items[(df_basket_items['序號']==2513) & (df_basket_items['銷售日期']=='2018-04-02') & (df_basket_items['機台']==66)]

In [ ]:
df_ele_data[(df_ele_data['序號']==2513) & (df_ele_data['銷售日期']=='2018/4/2')]

In [ ]:
df_credit_card[(df_credit_card['序號']==2513) & (df_credit_card['銷售日期']=='2018-04-02')]

處理member_card

In [ ]:
df_basket_items[(df_basket_items['序號']==1066) & (df_basket_items['銷售日期']=='2018-04-02') & (df_basket_items['機台']==66)]

In [ ]:
df_member_card[(df_member_card['交易序號']==1066) & (df_member_card['日期']=='2018/4/2') & (df_member_card['機台']==585066)]

## 結論：
* 用【銷售日期+機台+序號】合併表格
* 機台需要再+5850

In [ ]:
df_basket_items.info()

## 購物籃分析

In [ ]:
df_basket_items = (
    df_basket_items
    .assign(
        # Rebuild basket id column
        購物籃編號 = (
        df_basket_items['銷售日期'].dt.strftime('%Y%m%d')
        + '5850'
        + df_basket_items['機台'].astype(str)
        + df_basket_items['序號'].astype(str)
        ),
        # Summerize total amount
        購物籃營收 = (
        df_basket_items
        .groupby('購物籃編號')['總金額']
        .transform('sum') 
    ))
    .sort_values(by='購物籃營收',ascending=False) # Sort values
    # .head() # Un-comment if needed
)

# df_basket_items['購物籃編號'] = df_basket_items['銷售日期'].dt.strftime('%Y%m%d') + '5850' + df_basket_items['機台'].astype(str) + df_basket_items['序號'].astype(str)
# df_basket_items['購物籃營收'] = df_basket_items.groupby('購物籃編號')['總金額'].transform('sum')
# df_basket_items.sort_values(by='購物籃營收',ascending=False)

In [ ]:
# df_basket_items.head()

找出最高營收的購物籃

In [ ]:
revenue_per_basket = (
    df_basket_items
    .groupby('購物籃編號')['總金額']
    .sum()
    .to_frame()
    .sort_values(by='總金額',ascending=False)
    .head(15)
    .select()
    .loc[df_basket_items['購物籃編號'] == '20180423585047833']
    )

# df_revenue_per_basket = revenue_per_basket.to_frame()
# df_revenue_per_basket.sort_values(by='總金額',ascending=False).head(15)

In [ ]:
# df_basket_items[df_basket_items['購物籃編號'] == '20180423585047833']

## Apriori演算法

In [ ]:
columns_to_keep = ['短品名', '購物籃編號']
df_apriori = df_basket_items.filter(columns_to_keep)

In [ ]:
df_apriori

In [ ]:
df_apriori = (
    df_apriori
    .groupby('購物籃編號')['短品名']
    .apply(list)
    .reset_index(name='品項')
    )
df_apriori['品項'] = df_apriori['品項'].apply(lambda x: list(set(x)))  # 使用集合去除重複值
df_apriori

In [ ]:
transactions = df_apriori['品項'].tolist()
# 把一些附屬品刪除
transactions = [[item for item in sublist if item not in ['瑞仕鑽石鍋印花', 'ＪＯ全店印花','北市環保兩用大']] for sublist in transactions]

參數還要再進行調整

In [ ]:
from apyori import apriori
rules = apriori(
    transactions = transactions,
    min_support = 0.002,
    min_confidence = 0.01,
    min_lift = 2,
    min_length = 2,
    max_length = 2)
results = list(rules)
results

In [ ]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))

resultsinDataFrame = pd.DataFrame(
    inspect(results),
    columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift']
    )
resultsinDataFrame.sort_values(by='Lift',ascending=False)